In [1]:
!pip install -U trl
!pip install -U transformers
!pip install -U accelerate
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import LoraConfig, TaskType
from trl import RewardTrainer, RewardConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


In [2]:
model_id = "HuggingFaceTB/smollm2-135M-SFT-Only"
dataset_name = "Intel/orca_dpo_pairs"
output_dir = "smollm2-reward-model"

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=1,
    device_map="auto"
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/smollm2-135M-SFT-Only and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

In [4]:
dataset = load_dataset(dataset_name, split="train")

def format_for_reward_model(example):
    prompt_messages = [
        {"role": "system", "content": example["system"]},
        {"role": "user", "content": example["question"]}
    ]
    prompt_text = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
    chosen_text = prompt_text + "\nAssistant: " + example["chosen"]
    rejected_text = prompt_text + "\nAssistant: " + example["rejected"]
    return {
        "chosen": chosen_text,
        "rejected": rejected_text
    }

train_dataset = dataset.map(format_for_reward_model, num_proc=4)

README.md:   0%|          | 0.00/196 [00:00<?, ?B/s]

orca_rlhf.jsonl:   0%|          | 0.00/36.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12859 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/12859 [00:00<?, ? examples/s]

In [5]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear"
)

training_args = RewardConfig(
    output_dir=output_dir,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    max_length=1024,
    center_rewards_coefficient=0.01
)

trainer = RewardTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    peft_config=peft_config,
)

Adding EOS to train dataset:   0%|          | 0/12859 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/12859 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2084 > 2048). Running this sequence through the model will result in indexing errors


Filtering train >1024 tokens:   0%|          | 0/12859 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [6]:
print("Starting Reward Model training...")
trainer.train()

trainer.save_model(output_dir + "-final")
print("Reward Model training complete.")

Starting Reward Model training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 27100046 (27100046-lahore-university-of-management-sciences) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.282400
20,1.072100
30,0.796300
40,0.600800
50,0.452500
60,0.374100
70,0.256000
80,0.200300
90,0.121500
100,0.070000


Reward Model training complete.


In [7]:
from google.colab import files
import shutil

shutil.make_archive('smollm2-reward-model-final', 'zip', './smollm2-reward-model-final')

files.download('smollm2-reward-model-final.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>